In [81]:
import pandas as pd

def load_data(file_path):
    try:
        data = pd.read_csv(file_path)
        return data
    except Exception as e:
        print(f"Error loading data: {e}")
        return None

base_path = 'datasets/transfermarkt/'
clubs = load_data(base_path + 'clubs.csv')
competitions = load_data(base_path + 'competitions.csv')
transfers = load_data(base_path + 'transfers.csv')
games = load_data(base_path + 'games.csv')

In [82]:
print(games.columns)

Index(['game_id', 'competition_id', 'season', 'round', 'date', 'home_club_id',
       'away_club_id', 'home_club_goals', 'away_club_goals',
       'home_club_position', 'away_club_position', 'home_club_manager_name',
       'away_club_manager_name', 'stadium', 'attendance', 'referee', 'url',
       'home_club_formation', 'away_club_formation', 'home_club_name',
       'away_club_name', 'aggregate', 'competition_type'],
      dtype='object')


In [83]:
games['season'].value_counts().sort_index()


2012    5700
2013    5762
2014    5836
2015    5708
2016    5699
2017    5596
2018    5723
2019    5469
2020    5513
2021    5875
2022    6008
2023    5839
2024    5298
Name: season, dtype: int64

In [84]:
games['home_club_id'].value_counts().sort_values()


32218.0      1
8303.0       1
4324.0       1
60930.0      1
29817.0      1
          ... 
281.0      344
631.0      349
506.0      352
131.0      361
418.0      368
Name: home_club_id, Length: 2542, dtype: int64

In [85]:
print(clubs.columns)


Index(['club_id', 'club_code', 'name', 'domestic_competition_id',
       'total_market_value', 'squad_size', 'average_age', 'foreigners_number',
       'foreigners_percentage', 'national_team_players', 'stadium_name',
       'stadium_seats', 'net_transfer_record', 'coach_name', 'last_season',
       'filename', 'url'],
      dtype='object')


In [86]:
print(transfers.columns)

Index(['player_id', 'transfer_date', 'transfer_season', 'from_club_id',
       'to_club_id', 'from_club_name', 'to_club_name', 'transfer_fee',
       'market_value_in_eur', 'player_name'],
      dtype='object')


In [87]:
import pandas as pd
import numpy as np
import plotly.express as px

# ---------------------------------------------
# Helper functions
# ---------------------------------------------
def _clubs_with_country(clubs, competitions):
    """Attach each club's domestic competition country."""
    cc = competitions[['competition_id', 'country_name']].rename(
        columns={'competition_id': 'domestic_competition_id', 'country_name': 'club_country'}
    )
    return clubs.merge(cc, on='domestic_competition_id', how='left')

def _top_30_clubs_by_games(games, clubs):
    home_counts = games.groupby('home_club_id').size().rename('home_games')
    away_counts = games.groupby('away_club_id').size().rename('away_games')
    total = pd.concat([home_counts, away_counts], axis=1).fillna(0)
    total['total_games'] = total['home_games'] + total['away_games']
    total = total.sort_values('total_games', ascending=False)
    top30_ids = total.head(30).index.astype(int).tolist()
    return top30_ids

def _prep_transfers(transfers):
    t = transfers.copy()
    t['transfer_fee'] = pd.to_numeric(t['transfer_fee'], errors='coerce').fillna(0)
    return t

def _pivot_order_columns_by_total(df, index_name, columns_name, values_name):
    totals = df.groupby(columns_name)[values_name].sum().sort_values(ascending=False)
    countries_sorted = totals.index.tolist()
    pivot = df.pivot(index=index_name, columns=columns_name, values=values_name).fillna(0)
    pivot = pivot.reindex(columns=[c for c in countries_sorted if c in pivot.columns])
    return pivot

def _imshow_grouped(df, clubs_cc, title, xlab, ylab, zlab):
    """
    df: pivot with club names as the index
    clubs_cc: DataFrame with at least ['name','club_country'] for those clubs
    """
    # Keep 'name' as a column while sorting; only set as index at the end
    order_df = (
        clubs_cc.loc[clubs_cc['name'].isin(df.index), ['name', 'club_country']]
                 .copy()
                 .assign(club_country=lambda d: d['club_country'].fillna('Unknown'))
                 .sort_values(['club_country', 'name'])
                 .set_index('name')
    )

    # Reindex the heatmap rows by grouped (country, name) order
    ordered_clubs = order_df.index.tolist()
    df = df.reindex(index=ordered_clubs)

    # Build the heatmap
    fig = px.imshow(
        df.fillna(0),
        x=df.columns,
        y=df.index,
        labels=dict(x=xlab, y=ylab, color=zlab),
        title=title
    )

    # --- Force ALL x tick labels (every country) ---
    fig.update_xaxes(
        tickmode='array',
        tickvals=list(range(len(df.columns))),
        ticktext=df.columns.tolist(),
        tickangle=-45
    )
    # Make the figure wide enough so labels don’t overlap
    fig.update_layout(width=max(900, 30 * len(df.columns)))

    # Optional: show country next to club for clarity
    y_text = [f"{club} ({order_df.loc[club, 'club_country']})" for club in df.index]
    fig.update_yaxes(
        tickvals=list(range(len(y_text))),
        ticktext=y_text,
        categoryorder='array',
        categoryarray=df.index.tolist()
    )

    # Draw separators between countries
    y_positions = []
    last_country = None
    for i, club in enumerate(order_df.index):
        country = order_df.loc[club, 'club_country']
        if last_country is None:
            last_country = country
        elif country != last_country:
            y_positions.append(i - 0.5)
            last_country = country
    for y in y_positions:
        fig.add_shape(
            type="line",
            x0=-0.5, x1=len(df.columns)-0.5,
            y0=y, y1=y,
            line=dict(color="white", width=2)
        )

    fig.update_layout(
        yaxis_title=None,
        xaxis_title=None,
        margin=dict(l=180, r=20, t=80, b=40)
    )
    fig.show()



# ---------------------------------------------
# Main analysis function
# ---------------------------------------------
def heatmaps_for_top30_grouped_by_country(
    games, clubs, competitions, transfers, season_filter=None
):
    # Step 1: Filter seasons if desired
    if season_filter is not None:
        games = games[games['season'].isin(season_filter)]
        transfers = transfers[transfers['transfer_season'].isin(season_filter)]

    # Step 2: Attach club countries
    clubs_cc = _clubs_with_country(clubs, competitions)

    # Step 3: Top 30 clubs by number of games
    top30_ids = _top_30_clubs_by_games(games, clubs)
    top30_clubs = clubs_cc[clubs_cc['club_id'].isin(top30_ids)].copy()
    top30_names = top30_clubs['name'].tolist()

    # Step 4: Prepare transfer data
    transfers = _prep_transfers(transfers)
    club_countries = clubs_cc[['club_id', 'club_country']].rename(columns={'club_country': 'country'})
    from_cc = club_countries.rename(columns={'club_id': 'from_club_id', 'country': 'from_country'})
    to_cc = club_countries.rename(columns={'club_id': 'to_club_id', 'country': 'to_country'})
    transfers = transfers.merge(from_cc, on='from_club_id', how='left')
    transfers = transfers.merge(to_cc, on='to_club_id', how='left')

    # Step 5: Split into incoming/outgoing transfers for top 30 clubs
    incoming = transfers[transfers['to_club_id'].isin(top30_ids)].copy()
    outgoing = transfers[transfers['from_club_id'].isin(top30_ids)].copy()
    incoming['club_name'] = incoming['to_club_id'].map(dict(zip(clubs['club_id'], clubs['name'])))
    outgoing['club_name'] = outgoing['from_club_id'].map(dict(zip(clubs['club_id'], clubs['name'])))

    # Step 6: Compute matrices
    spend_amt = incoming.groupby(['club_name', 'from_country'])['transfer_fee'].sum().reset_index()
    recv_count = incoming.groupby(['club_name', 'from_country']).size().reset_index(name='player_count')
    income_amt = outgoing.groupby(['club_name', 'to_country'])['transfer_fee'].sum().reset_index()
    sent_count = outgoing.groupby(['club_name', 'to_country']).size().reset_index(name='player_count')

    # Step 7: Pivot to matrices
    spend_amt_pivot = _pivot_order_columns_by_total(spend_amt, 'club_name', 'from_country', 'transfer_fee')
    recv_count_pivot = _pivot_order_columns_by_total(recv_count, 'club_name', 'from_country', 'player_count')
    income_amt_pivot = _pivot_order_columns_by_total(income_amt, 'club_name', 'to_country', 'transfer_fee')
    sent_count_pivot = _pivot_order_columns_by_total(sent_count, 'club_name', 'to_country', 'player_count')

    # Step 8: Heatmaps with country grouping
    _imshow_grouped(spend_amt_pivot, top30_clubs, "Amount Spent on Incoming Transfers (Grouped by Club Country)",
                    "From Country", "Club", "Total Spending (€)")
    _imshow_grouped(recv_count_pivot, top30_clubs, "Players Received (Grouped by Club Country)",
                    "From Country", "Club", "Players Received")
    _imshow_grouped(income_amt_pivot, top30_clubs, "Amount Received on Outgoing Transfers (Grouped by Club Country)",
                    "To Country", "Club", "Total Income (€)")
    _imshow_grouped(sent_count_pivot, top30_clubs, "Players Sent (Grouped by Club Country)",
                    "To Country", "Club", "Players Sent")

    return {
        "spend_amount": spend_amt_pivot,
        "received_count": recv_count_pivot,
        "income_amount": income_amt_pivot,
        "sent_count": sent_count_pivot
    }


# Example usage:
# season_filter = ['15/16','16/17','17/18','18/19','19/20','20/21','21/22','22/23','23/24','24/25']
results = heatmaps_for_top30_grouped_by_country(games, clubs, competitions, transfers)


In [88]:
import pandas as pd
import numpy as np

from bokeh.plotting import figure, show
from bokeh.models import (
    ColumnDataSource, LinearColorMapper, ColorBar, BasicTicker,
    HoverTool, FactorRange
)
from bokeh.transform import transform
from bokeh.palettes import Viridis256

# ---------------------------------------------------------
# Helpers (same logic as before; Bokeh-specific plotting)
# ---------------------------------------------------------
def _clubs_with_country(clubs, competitions):
    """Attach each club's domestic competition country as club_country."""
    cc = competitions[['competition_id', 'country_name']].rename(
        columns={'competition_id': 'domestic_competition_id', 'country_name': 'club_country'}
    )
    return clubs.merge(cc, on='domestic_competition_id', how='left')

def _top_50_clubs_by_games(games, clubs):
    """Return list of top 50 club_ids by (home+away) appearances."""
    home_counts = games.groupby('home_club_id').size().rename('home_games')
    away_counts = games.groupby('away_club_id').size().rename('away_games')
    total = pd.concat([home_counts, away_counts], axis=1).fillna(0)
    total['total_games'] = total['home_games'] + total['away_games']
    top50_ids = total.sort_values('total_games', ascending=False).head(50).index.astype(int).tolist()
    return top50_ids

def _prep_transfers(transfers):
    t = transfers.copy()
    t['transfer_fee'] = pd.to_numeric(t['transfer_fee'], errors='coerce').fillna(0)
    return t

def _pivot_order_columns_by_total(df, index_name, columns_name, values_name):
    """Make pivot; order columns (countries) by total descending."""
    totals = df.groupby(columns_name)[values_name].sum().sort_values(ascending=False)
    countries_sorted = totals.index.tolist()
    pivot = df.pivot(index=index_name, columns=columns_name, values=values_name).fillna(0)
    pivot = pivot.reindex(columns=[c for c in countries_sorted if c in pivot.columns])
    return pivot

# ---- Small utility we added later: keep top-K/threshold columns (countries) ----
def keep_top_columns(pivot_df, k=None, min_total=None):
    """
    Keep top-k countries by column total, or those >= min_total.
    If neither is provided, return unchanged.
    """
    totals = pivot_df.sum(axis=0).sort_values(ascending=False)
    if k is not None:
        keep = totals.head(k).index
        return pivot_df.loc[:, keep]
    if min_total is not None:
        keep = totals[totals >= min_total].index
        return pivot_df.loc[:, keep]
    return pivot_df

# ---------------------------------------------------------
# Bokeh heatmap (grouped by club country)
# ---------------------------------------------------------
def _bokeh_heatmap_grouped(df, clubs_cc, title, xlab, ylab, zlab, cell_size=25):
    """
    df: pivot with club names as index, countries as columns
    clubs_cc: DataFrame with ['name','club_country'] for those clubs
    """
    # Sort clubs by (club_country, name); reindex rows to that order
    order_df = (
        clubs_cc.loc[clubs_cc['name'].isin(df.index), ['name', 'club_country']]
                .copy()
                .assign(club_country=lambda d: d['club_country'].fillna('Unknown'))
                .sort_values(['club_country', 'name'])
                .set_index('name')
    )
    ordered_clubs = order_df.index.tolist()
    df = df.reindex(index=ordered_clubs).fillna(0)

    # Categorical axes: countries (y) and clubs (x) - FLIPPED
    x_factors = df.index.tolist()                 # club names
    y_factors = df.columns.tolist()               # countries
    # Show country in the label too
    x_label_overrides = {club: f"{club} ({order_df.loc[club, 'club_country']})" for club in x_factors}

    # Long form for rects - swap club and country
    mat_long = (
        df.reset_index()
          .melt(id_vars=df.index.name or 'index', var_name='country', value_name='value')
          .rename(columns={df.index.name or 'index': 'club'})
    )
    source = ColumnDataSource(mat_long)

    # Color mapping
    vmin = float(mat_long['value'].min())
    vmax = float(mat_long['value'].max())
    if vmin == vmax:
        vmax = vmin + 1e-9  # avoid degenerate color range
    mapper = LinearColorMapper(palette=Viridis256, low=vmin, high=vmax)

    # Dynamic sizing: FLIPPED - clubs on x-axis, countries on y-axis
    # Use same size for both dimensions to create square cells
    plot_width = max(900, int(cell_size * max(1, len(x_factors))) + 300)  # +300 for labels and margins
    plot_height = max(400, int(cell_size * max(1, len(y_factors))) + 150)  # +150 for title and margins

    p = figure(
        x_range=x_factors,
        y_range=FactorRange(*y_factors),
        x_axis_location="below",
        width=plot_width,
        height=plot_height,
        title=title,
        tools="pan,box_zoom,reset,save"
    )

    # Heatmap - swap x and y
    p.rect(
        x="club", y="country", width=1, height=1,
        source=source,
        line_color=None,
        fill_color=transform('value', mapper)
    )

    # Color bar
    color_bar = ColorBar(
        color_mapper=mapper,
        ticker=BasicTicker(desired_num_ticks=8),
        label_standoff=8, location=(0, 0)
    )
    p.add_layout(color_bar, 'right')

    # Tooltips - swap club and country
    p.add_tools(HoverTool(
        tooltips=[
            (ylab, "@club"),
            (xlab, "@country"),
            (zlab, "@value{0,0}"),
        ],
        mode='mouse'
    ))

    # Show ALL x tick labels (clubs) + angle for readability - FLIPPED
    p.xaxis.major_label_orientation = np.deg2rad(-45)  # -45 degrees
    p.xaxis.major_label_text_font_size = "9pt"
    p.xaxis.major_label_overrides = x_label_overrides
    
    # For y, countries
    p.yaxis.major_label_text_font_size = "10pt"

    # Layout tweaks - swap labels
    p.xaxis.axis_label = ylab
    p.yaxis.axis_label = xlab
    p.xaxis.axis_label_text_font_size = "12pt"
    p.yaxis.axis_label_text_font_size = "12pt"
    
    # Improve spacing
    p.min_border_left = 200
    p.min_border_bottom = 100

    show(p)

# ---------------------------------------------------------
# Main analysis function (Bokeh version)
# ---------------------------------------------------------
def bokeh_heatmaps_for_top50_grouped_by_country(
    games, clubs, competitions, transfers, games_season_filter=None, transfer_season_filter=None,
    topk_countries=None, min_total_per_country=None
):
    """
    Builds four heatmaps using Bokeh, grouping clubs by their domestic country.
    Optional filters:
      - topk_countries: keep only top K countries per heatmap
      - min_total_per_country: keep only countries whose column total >= threshold
    """
    # Optional season filtering
    if games_season_filter is not None and transfer_season_filter is not None:
        games = games[games['season'].isin(games_season_filter)]
        transfers = transfers[transfers['transfer_season'].isin(transfer_season_filter)]

    # Attach club countries
    clubs_cc = _clubs_with_country(clubs, competitions)

    # Top 50 clubs by matches
    top50_ids = _top_50_clubs_by_games(games, clubs)
    id_to_name = dict(zip(clubs_cc['club_id'], clubs_cc['name']))
    top50_clubs = clubs_cc.loc[clubs_cc['club_id'].isin(top50_ids), ['club_id','name','club_country']].copy()

    # Transfers + map club countries on both ends
    t = _prep_transfers(transfers)
    cc = clubs_cc[['club_id', 'club_country']]
    t = t.merge(cc.rename(columns={'club_id':'from_club_id','club_country':'from_country'}), on='from_club_id', how='left')
    t = t.merge(cc.rename(columns={'club_id':'to_club_id','club_country':'to_country'}), on='to_club_id',   how='left')

    # Subsets for the top-50
    incoming = t[t['to_club_id'].isin(top50_ids)].copy()
    outgoing = t[t['from_club_id'].isin(top50_ids)].copy()
    incoming['club_name'] = incoming['to_club_id'].map(id_to_name)
    outgoing['club_name'] = outgoing['from_club_id'].map(id_to_name)

    # Matrices
    spend_amt = (incoming.groupby(['club_name', 'from_country'], dropna=False)['transfer_fee']
                        .sum().reset_index().assign(from_country=lambda d: d['from_country'].fillna('Unknown')))
    recv_cnt  = (incoming.groupby(['club_name', 'from_country'], dropna=False)
                        .size().reset_index(name='player_count').assign(from_country=lambda d: d['from_country'].fillna('Unknown')))
    income_amt = (outgoing.groupby(['club_name', 'to_country'], dropna=False)['transfer_fee']
                        .sum().reset_index().assign(to_country=lambda d: d['to_country'].fillna('Unknown')))
    sent_cnt   = (outgoing.groupby(['club_name', 'to_country'], dropna=False)
                        .size().reset_index(name='player_count').assign(to_country=lambda d: d['to_country'].fillna('Unknown')))

    # Pivots ordered by total across columns
    spend_pv = _pivot_order_columns_by_total(spend_amt, 'club_name', 'from_country', 'transfer_fee')
    recv_pv  = _pivot_order_columns_by_total(recv_cnt,  'club_name', 'from_country', 'player_count')
    income_pv= _pivot_order_columns_by_total(income_amt,'club_name', 'to_country',   'transfer_fee')
    sent_pv  = _pivot_order_columns_by_total(sent_cnt,  'club_name', 'to_country',   'player_count')

    # ---- (small part) Optionally keep only the top-K / threshold countries ----
    if (topk_countries is not None) or (min_total_per_country is not None):
        spend_pv  = keep_top_columns(spend_pv,  k=topk_countries, min_total=min_total_per_country)
        recv_pv   = keep_top_columns(recv_pv,   k=topk_countries, min_total=min_total_per_country)
        income_pv = keep_top_columns(income_pv, k=topk_countries, min_total=min_total_per_country)
        sent_pv   = keep_top_columns(sent_pv,   k=topk_countries, min_total=min_total_per_country)

    # ---- (small part) Sanity check: list the countries that WILL appear in each heatmap ---
    print("Spending countries:",  spend_pv.columns.tolist())
    print("Received-from countries:", recv_pv.columns.tolist())
    print("Income countries:",    income_pv.columns.tolist())
    print("Sent-to countries:",   sent_pv.columns.tolist())

    # Titles
    suffix = "" if transfer_season_filter is None else f" (Seasons: {', '.join(transfer_season_filter)})"

    _bokeh_heatmap_grouped(
        spend_pv, top50_clubs,
        title=f"Amount Spent on Incoming Transfers by From-Country{suffix}",
        xlab="From Country", ylab="Club", zlab="Total Spend (€)"
    )

    _bokeh_heatmap_grouped(
        recv_pv, top50_clubs,
        title=f"Players Received by From-Country{suffix}",
        xlab="From Country", ylab="Club", zlab="# Players"
    )

    _bokeh_heatmap_grouped(
        income_pv, top50_clubs,
        title=f"Amount Received from Outgoing Transfers by To-Country{suffix}",
        xlab="To Country", ylab="Club", zlab="Total Income (€)"
    )

    _bokeh_heatmap_grouped(
        sent_pv, top50_clubs,
        title=f"Players Sent by To-Country{suffix}",
        xlab="To Country", ylab="Club", zlab="# Players"
    )

    return {
        "spend_amount": spend_pv,
        "received_count": recv_pv,
        "income_amount": income_pv,
        "sent_count": sent_pv
    }

# ---------------------------------------------------------
# Example usage
# ---------------------------------------------------------
# All seasons
results = bokeh_heatmaps_for_top50_grouped_by_country(games, clubs, competitions, transfers)

#Or restrict seasons (as in your earlier snippet)
# games_season_filter = [2015, 2016]
# transfer_season_filter = ['15/16']
# results = bokeh_heatmaps_for_top50_grouped_by_country(
#     games, clubs, competitions, transfers,
#     games_season_filter=games_season_filter,
#     transfer_season_filter=transfer_season_filter
#     # min_total_per_country=5_000_000   # alternative: threshold by total
# )


Spending countries: ['England', 'Italy', 'Spain', 'Germany', 'France', 'Unknown', 'Portugal', 'Netherlands', 'Belgium', 'Turkey', 'Russia', 'Ukraine', 'Denmark', 'Greece', 'Scotland']
Received-from countries: ['Unknown', 'Italy', 'England', 'Spain', 'France', 'Germany', 'Portugal', 'Netherlands', 'Turkey', 'Belgium', 'Greece', 'Russia', 'Scotland', 'Denmark', 'Ukraine']
Income countries: ['England', 'Italy', 'Spain', 'France', 'Germany', 'Unknown', 'Portugal', 'Russia', 'Turkey', 'Netherlands', 'Belgium', 'Scotland', 'Ukraine', 'Greece', 'Denmark']
Sent-to countries: ['Italy', 'Unknown', 'England', 'Spain', 'Germany', 'France', 'Portugal', 'Turkey', 'Netherlands', 'Greece', 'Belgium', 'Russia', 'Scotland', 'Denmark', 'Ukraine']
